In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Taxi ETL") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()

26/01/09 19:24:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
df=spark.read.parquet('../data/dataset.parquet')
df.show(5)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [8]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [9]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [10]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'cbd_congestion_fee']

In [11]:
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'bigint'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('Airport_fee', 'double'),
 ('cbd_congestion_fee', 'double')]

In [12]:
df.describe().show()

26/01/09 19:24:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+------------------+--------------------+-------------------+-------------------+
|summary|          VendorID|   passenger_count|    trip_distance|       RatecodeID|store_and_fwd_flag|     PULocationID|      DOLocationID|      payment_type|       fare_amount|             extra|            mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|        Airport_fee| cbd_congestion_fee|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+-------

In [13]:
from pyspark.sql.functions import col, count, when

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       0|                   0|                    0|         540149|            0|    540149|            540149|           0|    

In [14]:
df_clean = df.na.drop()


In [15]:
df_clean.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       0|                   0|                    0|              0|            0|         0|                 0|           0|    

In [16]:
from pyspark.sql.functions import unix_timestamp

# Convertir les timestamps en secondes et faire la différence
df_clean = df_clean.withColumn(
    "trip_duration",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
)

# df_clean.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_duration").show()
# Afficher les lignes avec trip_duration <= 0
df_clean.filter(col("trip_duration") <= 0).count()

1832

In [17]:
df_clean = df_clean.filter(col("trip_duration") > 0)
df_clean.show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|     trip_duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|    

In [18]:
df_clean = df_clean.filter((col("passenger_count").isNotNull()) & (col("passenger_count") > 0))
df_clean.show()


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|     trip_duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|    

In [19]:
df_clean = df_clean.drop("duration")

df_clean.show(10)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|     trip_duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|    

In [20]:
# Verification du nombre de colonnes après nettoyage
len(df_clean.columns)


21

#### Identification des colonnes de types de données

In [21]:
#Get All column names and it's types
for col in df_clean.dtypes:
    print(col[0]+" , "+col[1])

VendorID , int
tpep_pickup_datetime , timestamp_ntz
tpep_dropoff_datetime , timestamp_ntz
passenger_count , bigint
trip_distance , double
RatecodeID , bigint
store_and_fwd_flag , string
PULocationID , int
DOLocationID , int
payment_type , bigint
fare_amount , double
extra , double
mta_tax , double
tip_amount , double
tolls_amount , double
improvement_surcharge , double
total_amount , double
congestion_surcharge , double
Airport_fee , double
cbd_congestion_fee , double
trip_duration , double


In [22]:
#store all column names in a list
allCols = [item[0] for item in df_clean.dtypes]

In [23]:
# Colonnes catégorielles : string + certaines colonnes bigint que tu sais être des catégories
categoricalCols = [c for c, t in df_clean.dtypes if t.startswith('string')] + ["payment_type", "RatecodeID", "VendorID"]
print(categoricalCols)

['store_and_fwd_flag', 'payment_type', 'RatecodeID', 'VendorID']


In [24]:
# Colonnes continues : int/bigint/double qui sont des nombres réels à calculer
numsCols = [c for c, t in df_clean.dtypes if t in ('double','int','bigint') and c not in categoricalCols]
print(numsCols)

['passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee', 'cbd_congestion_fee', 'trip_duration']


In [25]:
from pyspark.sql import functions as F
from functools import reduce

def find_outliers(df, numerical_columns=None):
    """
    Ajoute des colonnes pour marquer les outliers et compte le total
    """
    if numerical_columns is None:
        numeric_columns = [c for c,t in df.dtypes if t in ('int','bigint','double')]
    else:
        numeric_columns = numerical_columns

    for column in numeric_columns:
        Q1 = df.approxQuantile(column,[0.25],0)[0]
        Q3 = df.approxQuantile(column,[0.75],0)[0]
        IQR = Q3 - Q1

        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR

        df = df.withColumn(f'is_outlier_{column}', F.when((F.col(column) < lower) | (F.col(column) > upper), 1).otherwise(0))

    selected_columns = [c for c in df.columns if c.startswith("is_outlier")]
    df = df.withColumn('total_outliers', reduce(lambda a,b: a+b, [F.col(c) for c in selected_columns]))
    df = df.drop(*selected_columns)

    return df

- Les colonnes is_outlier_* sont supprimées pour garder un dataframe propre, mais total_outliers reste.
- Utilisation de la fonction Outlier personnalisée et application à une trame de données Spark


In [26]:
df_clean = find_outliers(df_clean, numsCols)
df_clean.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+--------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|     trip_duration|total_outliers|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+------------------+--------------+
|       1| 2025-01-01 00:18:

In [27]:
from pyspark.sql import functions as F

df_silver = df_clean.filter(
    (F.col("trip_duration") > 0) &
    (F.col("trip_distance") > 0) &
    (F.col("trip_distance") <= 200) &
    (F.col("passenger_count") > 0) &
    (F.col("passenger_count") <= 8) & 
    (F.col("fare_amount") >= 0) &
    (F.col("extra") >= 0) &
    (F.col("mta_tax") >= 0) &
    (F.col("tip_amount") >= 0) &
    (F.col("tolls_amount") >= 0) &
    (F.col("improvement_surcharge") >= 0) &
    (F.col("total_amount") >= 0) &
    (F.col("congestion_surcharge") >= 0) &
    (F.col("Airport_fee") >= 0) &
    (F.col("cbd_congestion_fee") >= 0)
)


Ajustement de la trame de données ci-dessus, pour sélectionner uniquement les enregistrements où le nombre de valeurs aberrantes est < = 1

In [28]:
# ===== DÉTECTER LES OUTLIERS AVEC IQR =====

# new_df_with_no_outliers = df_silver.filter(df_silver['total_outliers'] <= 1)
# new_df_with_no_outliers = new_df_with_no_outliers.drop('total_outliers')
# new_df_with_no_outliers.show()

print("\n=== DÉTECTION DES OUTLIERS (IQR) ===")
df_with_outliers = find_outliers(df_clean, numsCols)

# Voir la distribution des outliers
print("\n=== DISTRIBUTION DES OUTLIERS PAR LIGNE ===")
df_with_outliers.groupBy('total_outliers').count().orderBy('total_outliers').show()




=== DÉTECTION DES OUTLIERS (IQR) ===



=== DISTRIBUTION DES OUTLIERS PAR LIGNE ===


+--------------+-------+
|total_outliers|  count|
+--------------+-------+
|             0|1887285|
|             1| 545734|
|             2|  57229|
|             3|  40099|
|             4|  53625|
|             5|  95551|
|             6| 103717|
|             7|  85609|
|             8|  31689|
|             9|   6548|
|            10|   1482|
|            11|    211|
|            12|      1|
+--------------+-------+



In [29]:
df_silver = df_with_outliers.filter(F.col('total_outliers') <= 4)

In [30]:
df_silver.count()

2583972

Converting a spark dataframe into pandas dataframe

In [31]:
df_silver.printSchema()



root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)
 |-- trip_duration: double (nullable = true)
 |-- total_outliers: integer (n

In [32]:
# from sqlalchemy import create_engine


# DB_HOST="localhost"
# DB_PORT="5432"
# DB_NAME="taxi_nyc"
# DB_USER="taxi_user"
# DB_PASSWORD="manal.2480"

# engine = create_engine(
#     f"postgresql://{DB_USER}:{DB_PASSWORD}"
#     f"@{DB_HOST}:{DB_PORT}/{DB_NAME}"
# )

# df_test = df_silver.toPandas()
# df_test.to_sql("taxi_trips", engine, if_exists="replace", index=False)

# print("✅ Test OK!")


L'ensemble de données, qui contient 2 valeurs aberrantes ou plus dans chaque enregistrement


In [33]:
data_with_outliers = df_silver.filter(df_silver['total_Outliers']>=2)
data_with_outliers.count()

150953

In [34]:
# # Selecting the numerical columns from the original dataframe and converting into pandas
numsCols

['passenger_count',
 'trip_distance',
 'PULocationID',
 'DOLocationID',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'cbd_congestion_fee',
 'trip_duration']

In [35]:
# # Test de connexion simple
# import psycopg2

# try:
#     conn = psycopg2.connect(
#         host="postgres",
#         port=5432,
#         database="taxi",
#         user="taxi",
#         password="taxi"
#     )
#     print("✓ Connexion PostgreSQL réussie!")
#     conn.close()
# except Exception as e:
#     print(f"✗ Erreur de connexion: {e}")

In [36]:
# # Étape 3 : Sauvegarde PostgreSQL

# df.write \
#     .format("jdbc") \
#     .option("url", "jdbc:postgresql://postgres:5432/taxi") \
#     .option("dbtable", "silver_taxi_trips") \
#     .option("user", "taxi") \
#     .option("password", "taxi") \
#     .option("driver", "org.postgresql.Driver") \
#     .mode("overwrite") \
#     .save()

# print("✓ Données sauvegardées avec succès dans PostgreSQL!")



In [37]:
df_silver

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double, cbd_congestion_fee: double, trip_duration: double, total_outliers: int]

#### PRÉPARATION DES DONNÉES

In [38]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, UnivariateFeatureSelector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# Enlever la colonne total_outliers si elle existe encore
if 'total_outliers' in df_silver.columns:
    df_ml = df_silver.drop('total_outliers')
else:
    df_ml = df_silver

print(f"Dataset: {df_ml.count():,} lignes")

# Split train/test
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"Train: {train_df.count():,}")
print(f"Test:  {test_df.count():,}")


Dataset: 2,583,972 lignes


Train: 2,067,665


Test:  516,307


#### DÉFINITION DES COLONNES 

In [39]:
cat_cols = ["PULocationID", "DOLocationID", "RatecodeID", "VendorID", "payment_type"]
num_cols = [
    'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
    'congestion_surcharge', 'Airport_fee', 'cbd_congestion_fee'
]
target = "trip_duration"

print(f"\nFeatures:")
print(f"  - Catégorielles: {len(cat_cols)}")
print(f"  - Numériques: {len(num_cols)}")
print(f"  - Target: {target}")


Features:
  - Catégorielles: 5
  - Numériques: 12
  - Target: trip_duration


#### PIPELINE ML

In [40]:
# # 1. StringIndexer pour les catégories
# indexers = [
#     StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
#     for c in cat_cols
# ]

# # 2. OneHotEncoder
# encoders = [
#     OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_ohe")
#     for c in cat_cols
# ]

# # 3. VectorAssembler
# feature_cols = num_cols + [f"{c}_ohe" for c in cat_cols]
# assembler = VectorAssembler(
#     inputCols=feature_cols, 
#     outputCol="features", 
#     handleInvalid="skip"
# )

# # 4. Sélection des meilleures features
# selector = UnivariateFeatureSelector(
#     featuresCol="features",
#     outputCol="selectedFeatures",
#     labelCol=target,
#     selectionMode="numTopFeatures",
#     selectionThreshold=20  # Top 20 features
# )

# # 5. Modèle Gradient Boosted Trees
# gbt = GBTRegressor(
#     featuresCol="selectedFeatures",
#     labelCol=target,
#     maxIter=50,  # Réduit pour aller plus vite
#     maxDepth=5,
#     seed=42
# )

# # 6. Pipeline complet
# pipeline = Pipeline(stages=indexers + encoders + [assembler, selector, gbt])


#### ENTRAÎNEMENT

In [41]:
print("\n🚀 Entraînement du modèle...")
import time
start = time.time()

pipeline_model = pipeline.fit(train_df)

train_time = time.time() - start
print(f"✅ Entraînement terminé en {train_time:.2f}s ({train_time/60:.2f} min)")



🚀 Entraînement du modèle...


NameError: name 'pipeline' is not defined

#### PRÉDICTIONS

In [ ]:
print("\nPrédictions sur le test set...")
predictions = pipeline_model.transform(test_df)

#### ÉVALUATION

In [ ]:
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("\n" + "="*60)
print("RÉSULTATS SUR LE TEST SET")
print("="*60)
print(f"RMSE (Root Mean Squared Error): {rmse:.2f} minutes")
print(f"MAE  (Mean Absolute Error):     {mae:.2f} minutes")
print(f"R²   (Coefficient):              {r2:.4f}")
print("="*60)

# ===== FEATURES SÉLECTIONNÉES =====

selected_indices = pipeline_model.stages[-2].selectedFeatures
print(f"\nTop {len(selected_indices)} features sélectionnées:")
print(f"   Indices: {selected_indices}")

# ===== QUELQUES PRÉDICTIONS =====

print("\nExemples de prédictions:")
predictions.select(target, "prediction").show(10, False)

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
import time

# ===== 1. FEATURE ENGINEERING =====

print("🔧 Feature Engineering...")

# Extraire date/heure du pickup
df_ml = df_silver.withColumn("pickup_hour", F.hour("tpep_pickup_datetime")) \
                 .withColumn("pickup_dayofweek", F.dayofweek("tpep_pickup_datetime")) \
                 .withColumn("pickup_month", F.month("tpep_pickup_datetime"))

# Enlever total_outliers si présent
if 'total_outliers' in df_ml.columns:
    df_ml = df_ml.drop('total_outliers')

print(f"📊 Dataset: {df_ml.count():,} lignes")

# ===== 2. DÉFINIR LES BONNES FEATURES =====

# ⚠️ ON ENLÈVE fare_amount, total_amount, tip_amount, etc. (calculés APRÈS le trajet)
cat_cols = [
    "PULocationID",      # Lieu de départ
    "DOLocationID",      # Lieu d'arrivée
    "RatecodeID",        # Type de tarif
    "VendorID",          # Compagnie de taxi
    "payment_type",      # Mode de paiement
    "pickup_hour",       # Heure du pickup
    "pickup_dayofweek",  # Jour de la semaine
    "pickup_month"       # Mois
]

num_cols = [
    'passenger_count',   # Nombre de passagers
    'trip_distance'      # Distance (la plus importante!)
]

target = "trip_duration"

print(f"\n📋 Features:")
print(f"  ✅ Catégorielles: {cat_cols}")
print(f"  ✅ Numériques: {num_cols}")
print(f"  🎯 Target: {target}")

# ===== 3. SPLIT TRAIN/TEST =====

train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"\n📊 Train: {train_df.count():,} lignes")
print(f"📊 Test:  {test_df.count():,} lignes")

# ===== 4. PIPELINE ML =====

print("\n🔨 Construction du pipeline...")

# StringIndexer
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in cat_cols
]

# OneHotEncoder
encoders = [
    OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_ohe")
    for c in cat_cols
]

# VectorAssembler
feature_cols = num_cols + [f"{c}_ohe" for c in cat_cols]
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

# Modèle GBT avec meilleurs paramètres
gbt = GBTRegressor(
    featuresCol="features",
    labelCol=target,
    maxIter=100,        # Plus d'itérations
    maxDepth=6,         # Profondeur augmentée
    stepSize=0.1,       # Learning rate
    subsamplingRate=0.8,
    seed=42
)

# Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, gbt])

# ===== 5. ENTRAÎNEMENT =====

print("\n🚀 Entraînement du modèle...")
start = time.time()

pipeline_model = pipeline.fit(train_df)

train_time = time.time() - start
print(f"✅ Entraînement terminé en {train_time:.2f}s ({train_time/60:.2f} min)")

# ===== 6. PRÉDICTIONS =====

print("\n🔮 Prédictions...")
predictions = pipeline_model.transform(test_df)

# ===== 7. ÉVALUATION =====

evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("\n" + "="*70)
print("📈 RÉSULTATS SUR LE TEST SET")
print("="*70)
print(f"RMSE (Root Mean Squared Error): {rmse:.2f} minutes")
print(f"MAE  (Mean Absolute Error):     {mae:.2f} minutes")
print(f"R²   (Coefficient):              {r2:.4f}")
print("="*70)

# ===== 8. IMPORTANCE DES FEATURES =====

print("\n🔝 IMPORTANCE DES FEATURES:")
feature_importance = pipeline_model.stages[-1].featureImportances

# Créer un mapping des features
import numpy as np
importances = [(i, float(feature_importance[i])) for i in range(len(feature_importance))]
importances.sort(key=lambda x: x[1], reverse=True)

print("\nTop 10 features les plus importantes:")
for idx, imp in importances[:10]:
    if idx < len(num_cols):
        feat_name = num_cols[idx]
    else:
        feat_name = f"encoded_feature_{idx}"
    print(f"  {feat_name:30s}: {imp:.4f}")

# ===== 9. EXEMPLES DE PRÉDICTIONS =====

print("\n📋 EXEMPLES DE PRÉDICTIONS:")
predictions.select(
    "trip_distance",
    "passenger_count", 
    target,
    "prediction",
    (F.abs(F.col(target) - F.col("prediction"))).alias("erreur")
).show(15, False)

# ===== 10. ANALYSE DES ERREURS =====

print("\n📊 ANALYSE DES ERREURS:")
predictions_with_error = predictions.withColumn(
    "absolute_error",
    F.abs(F.col(target) - F.col("prediction"))
)

predictions_with_error.select("absolute_error").describe().show()
 
# Compter les prédictions dans différentes plages d'erreur
print("\nDistribution des erreurs:")
predictions_with_error.groupBy(
    F.when(F.col("absolute_error") < 2, "< 2 min")
     .when(F.col("absolute_error") < 5, "2-5 min")
     .when(F.col("absolute_error") < 10, "5-10 min")
     .otherwise("> 10 min")
     .alias("erreur_range")
).count().orderBy("erreur_range").show()

# # ===== 11. SAUVEGARDER =====

# print("\n💾 Sauvegarde du modèle...")
# pipeline_model.write().overwrite().save("model_taxi_duration_v2")
# print("✅ Modèle sauvegardé: model_taxi_duration_v2/")

print("\n🎉 TERMINÉ!")

🔧 Feature Engineering...


📊 Dataset: 2,583,972 lignes

📋 Features:
  ✅ Catégorielles: ['PULocationID', 'DOLocationID', 'RatecodeID', 'VendorID', 'payment_type', 'pickup_hour', 'pickup_dayofweek', 'pickup_month']
  ✅ Numériques: ['passenger_count', 'trip_distance']
  🎯 Target: trip_duration



📊 Train: 2,067,665 lignes


📊 Test:  516,307 lignes

🔨 Construction du pipeline...

🚀 Entraînement du modèle...


26/01/09 15:27:26 WARN MemoryStore: Not enough space to cache rdd_2074_10 in memory! (computed 99.6 MiB so far)
26/01/09 15:27:26 WARN BlockManager: Persisting block rdd_2074_10 to disk instead.
26/01/09 15:27:26 WARN MemoryStore: Not enough space to cache rdd_2074_6 in memory! (computed 66.0 MiB so far)
26/01/09 15:27:26 WARN BlockManager: Persisting block rdd_2074_6 to disk instead.
26/01/09 15:27:27 WARN MemoryStore: Not enough space to cache rdd_2074_2 in memory! (computed 236.1 MiB so far)
26/01/09 15:27:27 WARN BlockManager: Persisting block rdd_2074_2 to disk instead.
26/01/09 15:27:34 WARN MemoryStore: Not enough space to cache rdd_2074_10 in memory! (computed 236.1 MiB so far)
26/01/09 15:27:34 WARN MemoryStore: Not enough space to cache rdd_2074_10 in memory! (computed 44.0 MiB so far)
26/01/09 15:27:37 WARN MemoryStore: Not enough space to cache rdd_2074_2 in memory! (computed 66.0 MiB so far)
26/01/09 15:27:37 WARN MemoryStore: Not enough space to cache rdd_2074_2 in memory

...

+-------------+---------------+------------------+------------------+-------------------+
|trip_distance|passenger_count|trip_duration     |prediction        |erreur             |
+-------------+---------------+------------------+------------------+-------------------+
|5.6          |1              |15.083333333333334|14.511110630452535|0.5722227028807989 |
|1.2          |4              |8.733333333333333 |9.538229655118187 |0.8048963217848542 |
|0.5          |1              |2.65              |5.845096586667885 |3.195096586667885  |
|2.2          |1              |11.516666666666667|12.745428535488688|1.228761868822021  |
|4.2          |1              |24.033333333333335|21.640156837341816|2.3931764959915185 |
|0.9          |4              |7.533333333333333 |6.611303063263876 |0.9220302700694569 |
|1.0          |1              |8.466666666666667 |6.56871138279976  |1.8979552838669065 |
|5.4          |1              |23.05             |21.49916175322439 |1.550838246775612  |
|0.4      

26/01/09 16:58:23 WARN Executor: Issue communicating with driver in heartbeater 
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1324)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:322)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:194

+-------+--------------------+
|summary|      absolute_error|
+-------+--------------------+
|  count|              516307|
|   mean|  3.3399959607530563|
| stddev|  27.140320620641212|
|    min|1.020091377057497E-5|
|    max|   3391.691882743552|
+-------+--------------------+


Distribution des erreurs:


+------------+------+
|erreur_range| count|
+------------+------+
|     2-5 min|188844|
|    5-10 min| 57842|
|     < 2 min|256099|
|    > 10 min| 13522|
+------------+------+


🎉 TERMINÉ!


26/01/09 16:58:33 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1324)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:322)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.scala:18)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:1941

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkOK") \
    .getOrCreate()

spark.range(10).show()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/09 17:44:04 WARN Utils: Your hostname, zoro, resolves to a loopback address: 127.0.1.1; using 192.168.11.158 instead (on interface wlp1s0)
26/01/09 17:44:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/09 17:44:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [45]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double, cbd_congestion_fee: double]

In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# ===== 1. FEATURES TEMPORELLES + SPEED =====
print("Création des features...")

df_prepared = df_silver \
    .withColumn("pickuphour", F.hour("tpep_pickup_datetime")) \
    .withColumn("dayof_week", F.dayofweek("tpep_pickup_datetime")) \
    .withColumn("speed", F.col("trip_distance") / (F.col("trip_duration") / 60))

print(f"Avant filtrage: {df_prepared.count():,} lignes")

# ===== 2. FILTRAGE STRICT =====
print("\nFiltrage des valeurs aberrantes...")

df_clean = df_prepared.filter(
    (F.col("trip_distance") > 0.1) &
    (F.col("trip_distance") < 50) &
    (F.col("trip_duration") > 1) &
    (F.col("trip_duration") < 120) &
    (F.col("passenger_count") > 0) &
    (F.col("passenger_count") <= 6) &
    (F.col("speed") > 1) &          # Vitesse > 1 mph
    (F.col("speed") < 60)           # Vitesse < 60 mph
)

print(f"Après filtrage: {df_clean.count():,} lignes")

# ===== 3. PLUS DE FEATURES =====
num_cols = [
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "tolls_amount",
    "total_amount",
    "Airport_fee",
    "passenger_count",      # ⭐ NOUVEAU
    "extra",                # ⭐ NOUVEAU
    "mta_tax",              # ⭐ NOUVEAU
    "congestion_surcharge", # ⭐ NOUVEAU
    "speed"                 # ⭐ NOUVEAU
]

cat_cols = [
    "RatecodeID",
    "pickuphour",
    "dayof_week",
    "VendorID",        # ⭐ NOUVEAU
    "PULocationID",    # ⭐ NOUVEAU
    "DOLocationID",    # ⭐ NOUVEAU
    "payment_type"     # ⭐ NOUVEAU
]

target = "trip_duration"

print(f"\nFeatures numériques: {len(num_cols)}")
print(f"Features catégorielles: {len(cat_cols)}")

# ===== 4. SPLIT TRAIN/TEST =====
train_df, test_df = df_clean.randomSplit([0.8, 0.2], seed=42)

print(f"\nTrain: {train_df.count():,} lignes")
print(f"Test:  {test_df.count():,} lignes")

# ===== 5. PIPELINE ML =====
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in cat_cols
]

encoders = [
    OneHotEncoder(inputCol=f"{c}_idx", outputCol=f"{c}_ohe")
    for c in cat_cols
]

feature_cols = num_cols + [f"{c}_ohe" for c in cat_cols]
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
    handleInvalid="skip"
)

gbt = GBTRegressor(
    featuresCol="features",
    labelCol=target,
    maxIter=100,     # ⭐ AVANT: 50
    maxDepth=6,      # ⭐ AVANT: 5
    seed=42
)

pipeline = Pipeline(stages=indexers + encoders + [assembler, gbt])

# ===== 6. ENTRAÎNEMENT =====
print("\n🚀 Entraînement du modèle...")


pipeline_model = pipeline.fit(train_df)


# ===== 7. PRÉDICTIONS =====
predictions = pipeline_model.transform(test_df)

# ===== 8. ÉVALUATION =====
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print("\n" + "="*60)
print(f"RMSE: {rmse:.2f} minutes")
print(f"MAE:  {mae:.2f} minutes")
print(f"R²:   {r2:.4f} ")
print("="*60)


# ===== 10. SAUVEGARDER LE MODÈLE =====
pipeline_model.write().overwrite().save("/opt/airflow/models/eta_model")
print("\nModèle sauvegardé!")

Création des features...
Avant filtrage: 2,583,972 lignes

Filtrage des valeurs aberrantes...


Après filtrage: 2,539,180 lignes

Features numériques: 11
Features catégorielles: 7



Train: 2,031,809 lignes


Test:  507,371 lignes

🚀 Entraînement du modèle...


26/01/09 23:29:33 WARN MemoryStore: Not enough space to cache rdd_10690_10 in memory! (computed 234.4 MiB so far)
26/01/09 23:29:33 WARN BlockManager: Persisting block rdd_10690_10 to disk instead.
26/01/09 23:29:36 WARN MemoryStore: Not enough space to cache rdd_10690_2 in memory! (computed 155.5 MiB so far)
26/01/09 23:29:36 WARN BlockManager: Persisting block rdd_10690_2 to disk instead.
26/01/09 23:29:37 WARN MemoryStore: Not enough space to cache rdd_10690_6 in memory! (computed 234.4 MiB so far)
26/01/09 23:29:37 WARN BlockManager: Persisting block rdd_10690_6 to disk instead.
26/01/09 23:29:42 WARN MemoryStore: Not enough space to cache rdd_10690_10 in memory! (computed 234.4 MiB so far)
26/01/09 23:29:42 WARN MemoryStore: Not enough space to cache rdd_10690_10 in memory! (computed 43.7 MiB so far)
26/01/09 23:29:49 WARN MemoryStore: Not enough space to cache rdd_10690_2 in memory! (computed 351.7 MiB so far)
26/01/09 23:29:49 WARN MemoryStore: Not enough space to cache rdd_1069

✅ Terminé en 69.9 min



RMSE: 1.14 minutes
MAE:  0.51 minutes
R²:   0.9769 


+-------------+------------------+------------------+-------------------+
|trip_distance|trip_duration     |prediction        |erreur             |
+-------------+------------------+------------------+-------------------+
|5.6          |15.083333333333334|14.8875823683423  |0.19575096499103317|
|1.2          |8.733333333333333 |9.18111441875699  |0.44778108542365835|
|0.5          |2.65              |3.4494091603906436|0.7994091603906437 |
|2.2          |11.516666666666667|11.30762520479937 |0.20904146186729733|
|4.2          |24.033333333333335|23.957810133885662|0.07552319944767305|
|0.9          |7.533333333333333 |7.587175338540215 |0.05384200520688154|
|1.0          |8.466666666666667 |8.390024278980274 |0.07664238768639287|
|0.4          |2.183333333333333 |2.48391241793615  |0.3005790846028167 |
|2.2          |22.016666666666666|20.65146882699547 |1.3651978396711968 |
|0.6          |5.783333333333333 |6.402648292137978 |0.6193149588046447 |
+-------------+------------------+----